# Windowing Multi-Axis data tests

In this notebook, we check the windowing functions on multi-axis data.

## Step 1: Import libraries

In [1]:
import pandas as pd
import numpy as np
from caits import visualization as vis
from caits import windowing as wnd
from caits.dataset._dataset3 import CoreArray, DatasetArray, DatasetList

## Step 2: Initialize dataset

In [2]:
data = np.column_stack([np.arange(12), np.arange(100, 112)])
df = pd.DataFrame(data, columns=['feature_1', 'feature_2'])
df

,feature_1,feature_2
0,0,100
1,1,101
2,2,102
3,3,103
4,4,104
5,5,105
6,6,106
7,7,107
8,8,108
9,9,109


## Step 3: Rolling Window

In [3]:
rolling_wnd = wnd.rolling_window_df(df, ws=4, overlap=2)
print(len(rolling_wnd), rolling_wnd[0].shape)
rolling_wnd

5 (4, 2)


[   feature_1  feature_2
 0          0        100
 1          1        101
 2          2        102
 3          3        103,
    feature_1  feature_2
 2          2        102
 3          3        103
 4          4        104
 5          5        105,
    feature_1  feature_2
 4          4        104
 5          5        105
 6          6        106
 7          7        107,
    feature_1  feature_2
 6          6        106
 7          7        107
 8          8        108
 9          9        109,
     feature_1  feature_2
 8           8        108
 9           9        109
 10         10        110
 11         11        111]

## Step 4: Sliding window

In [4]:
sliding_wnd = wnd.sliding_window_df(df, 4, 2)
print(len(sliding_wnd), sliding_wnd[0].shape)
sliding_wnd

5 (4, 2)


[   feature_1  feature_2
 0          0        100
 1          1        101
 2          2        102
 3          3        103,
    feature_1  feature_2
 2          2        102
 3          3        103
 4          4        104
 5          5        105,
    feature_1  feature_2
 4          4        104
 5          5        105
 6          6        106
 7          7        107,
    feature_1  feature_2
 6          6        106
 7          7        107
 8          8        108
 9          9        109,
     feature_1  feature_2
 8           8        108
 9           9        109
 10         10        110
 11         11        111]

## Step 5: Frame signal

In [5]:
frames_wnd = wnd.frame_signal(data, 4, 2)
frames_wnd.shape

(4, 5, 2)

## Step 6: Create chunks

In [6]:
chunks = wnd.create_chunks(data, 5)
print(len(chunks), chunks[0].shape)
chunks

3 (5, 2)


[array([[  0, 100],
        [  1, 101],
        [  2, 102],
        [  3, 103],
        [  4, 104]]),
 array([[  5, 105],
        [  6, 106],
        [  7, 107],
        [  8, 108],
        [  9, 109]]),
 array([[ 10, 110],
        [ 11, 111]])]

# CaitsArray sliding window

In [7]:
data1 = np.concatenate([data, data, data, data, data, data, data, data], axis=0)

caitsArr = CoreArray(values=data1, axis_names={"axis_1": {"feature_1": 0, "feature_2": 1}})
caitsArr.iloc[:, 0]

  0    0
  1    1
  2    2
  3    3
  4    4
...  ...
 91    7
 92    8
 93    9
 94   10
 95   11

CaitsArray with shape (96,)

In [8]:
dataset = DatasetArray(X=caitsArr)
tmp = dataset.apply(wnd.sliding_window_arr, window_size=4, overlap=2)
tmp[0].shape

(4, 2)

In [9]:
tmp_stack = dataset.stack(tmp)
type(tmp_stack), len(tmp_stack), tmp_stack.X[0].shape

(caits.dataset._dataset3.DatasetList, 47, (4, 2))

In [10]:
caitsArr1 = CoreArray(values=data1[:, [0]])
caitsArr2 = CoreArray(values=data1[:, [1]])
dataset = DatasetList(X=[caitsArr1, caitsArr2])

In [11]:
tmp = dataset.apply(wnd.sliding_window_arr, window_size=4, overlap=2)
type(tmp), type(tmp[0]), type(tmp[0][0])


(list, list, numpy.ndarray)

In [12]:
tmp_stack = dataset.stack(tmp)
tmp_stack[0]

DatasetList object with 1 instances.

## Remove after check

In [13]:
from caits.filtering import filter_butterworth
from caits.preprocessing import normalize_signal
from caits.transformers._func_transformer_v2 import FunctionTransformer
from caits.transformers._sliding_window_v2 import SlidingWindow

f_hp = FunctionTransformer(filter_butterworth, fs=10, filter_type="highpass", cutoff_freq=3)
f_lp = FunctionTransformer(filter_butterworth, fs=10, filter_type="lowpass", cutoff_freq=3)
norm = FunctionTransformer(normalize_signal)
swnd = SlidingWindow(window_size=4, overlap=2)

In [14]:
tmp = f_lp.fit_transform(dataset)
tmp = f_hp.fit_transform(tmp)
tmp = norm.fit_transform(tmp)
tmp = swnd.fit_transform(tmp)
tmp.X[0].axis_names
# tmp.X[0].shape

{'axis_0': {0: 0, 1: 1, 2: 2, 3: 3}, 'axis_1': {0: 0}}